### Imports

In [667]:
import os
import numpy as np
import pandas as pd
from filterpy.kalman import KalmanFilter

### Inputs

In [668]:
fuzzy_output_df = pd.read_csv("../outputs/fuzzy_output.csv")
fuzzy_output_class = fuzzy_output_df['class'].values
fuzzy_output_confidence = fuzzy_output_df['confidence'].values

In [669]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df[["rssi", "etx", "delay", "busy_fraction"]].values

In [670]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_efsr_edge = radnet_input_df['efsr_edge'].values

In [671]:
confidence_fuzzy = np.where(
    fuzzy_output_class == 1,
    fuzzy_output_confidence,
    1.0 - fuzzy_output_confidence
)

### Kalman Filter

In [672]:
kf = KalmanFilter(dim_x=1, dim_z=1)

In [673]:
kf.x = np.array([confidence_fusion[0]])
kf.F = np.array([[1.]])
kf.H = np.array([[1.]])
kf.P = 1.0   # covariância inicial do estado
kf.R = 2e-2  # covariância da observação (measurement noise)
kf.Q = 4e-2  # covariância do processo (process noise)

In [674]:
kalman = []

for z in confidence_fusion:
    kf.predict()           
    kf.update(z)           
    kalman.append(kf.x[0])

In [675]:
kalman = np.array(kalman)

### Inference

In [676]:
cls = (kalman >= 0.5).astype(int)
conf = np.where(cls == 1, kalman, 1.0 - kalman)

In [677]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'EFSR':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*75}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{radnet_input_efsr_edge[i]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   EFSR   Classe   Status   Confiança 
---------------------------------------------------------------------------
-59    1.1   17     0.20   0.33   0        Ruim     0.61      
-60    1.4   6      0.10   0.91   1        Bom      0.61      
-78    6.8   19     0.49   0.91   1        Bom      0.53      
-84    11.0  18     0.26   0.36   0        Ruim     0.55      
-64    3.2   7      0.20   0.36   0        Ruim     0.58      
-43    1.5   4      0.32   0.90   1        Bom      0.60      
-58    1.3   4      0.22   0.90   1        Bom      0.65      
-57    1.2   14     0.17   0.90   1        Bom      0.66      
-62    1.0   7      0.11   0.93   1        Bom      0.70      
-69    1.8   5      0.22   0.38   1        Bom      0.50      
-62    3.1   7      0.34   0.32   0        Ruim     0.59      
-57    1.0   14     0.30   0.35   0        Ruim     0.59      
-49    2.6   18     0.19   0.41   0        Ruim     0.56      
-78    4.4   42     0.82   0.48   0       

### Save

In [678]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
